## Backtesting crypto arbitrage strategy

### Summary
In this notebook, we will try to find the best crypto pair and the best exchange pair 

### Go to project root

In [1]:
import os
from pathlib import Path

#list the current work dir
cwd = os.getcwd()
current_path = Path(cwd)
project_root = current_path.parent.parent

#change the current work dir
os.chdir(project_root)

### Imports

In [2]:
from collections import deque
from datetime import datetime, timedelta, timezone

from bot.event_loop import EventLoop
from broker.binance_fee_model import BinanceFeeModel
from broker.broker_manager import BrokerManager
from broker.kucoin_fee_model import KucoinFeeModel
from broker.simulated_broker import SimulatedBroker
from common.clock import SimulatedClock
from feed.feed import CsvFeed, Feed, ExternalStorageFeed
from indicators.indicators_manager import IndicatorsManager
from models.asset import Asset
from models.candle import Candle
from models.enums import Action, Direction, OrderType
from models.order import Order
from order_manager.order_creator import OrderCreator
from order_manager.order_manager import OrderManager
from order_manager.position_sizer import PositionSizer
from strategy.strategies.arbitrage_strategy import ArbitrageStrategy
from market_data.historical.binance_historical_data_handler import BinanceHistoricalDataHandler
from market_data.historical.kucoin_historical_data_handler import KucoinHistoricalDataHandler
from common.helper import get_or_create_nested_dict
from db_storage.mongodb_storage import MongoDbStorage
from db_storage.influxdb_storage import InfluxDbStorage

import numpy as np
import pandas as pd

### Define constants

In [32]:
BINANCE_EXCHANGE = "BINANCE"
KUCOIN_EXCHANGE = "KUCOIN"
EXCHANGES = [BINANCE_EXCHANGE, KUCOIN_EXCHANGE]
DATA_HANDLERS = {
    BINANCE_EXCHANGE: BinanceHistoricalDataHandler,
    KUCOIN_EXCHANGE: KucoinHistoricalDataHandler
}
FEE_MODELS = {BINANCE_EXCHANGE: BinanceFeeModel(), KUCOIN_EXCHANGE: KucoinFeeModel()}
seen = set()
EXCHANGE_PAIRS = []
for exchange1 in EXCHANGES:
    for exchange2 in EXCHANGES:
        if exchange2 in seen or exchange1 == exchange2:
            continue
        EXCHANGE_PAIRS.append((exchange1, exchange2))
    seen.add(exchange1)
LOOKBACK_PERIOD = timedelta(days=7)

STABLE_COINS = ["USDT", "USDC", "BUSD", "DAI", "UST", "TUSD", "PAX", "HUSD", "USDN", "GUSD"]
MINIMUM_TRANSACTIONS = 5
MINIMUM_VOLUME_IN_CASH = 1000

### Find common crypto pairs between exchange pairs

In [21]:
common_pairs_dict = {}
for exchange_pair in EXCHANGE_PAIRS:
    exchange1 = exchange_pair[0]
    exchange2 = exchange_pair[1]
    get_or_create_nested_dict(common_pairs_dict, exchange1, exchange2)
    
    exchange1_tickers_list = DATA_HANDLERS[exchange1].get_tickers_list()
    exchange2_tickers_list = DATA_HANDLERS[exchange2].get_tickers_list()
    
    common_pairs = np.intersect1d(exchange1_tickers_list, exchange2_tickers_list)
    
    def ends_with_stable_coin(pair: str):
        for stable_coin in STABLE_COINS:
            if pair.endswith(stable_coin):
                return True
        return False

    # Consider only stable coin pairs for now
    common_pairs = [common_pair for common_pair in common_pairs if ends_with_stable_coin(common_pair)]

    common_pairs_dict[exchange1][exchange2] = common_pairs

In [22]:
common_pairs_dict

{'BINANCE': {'KUCOIN': ['1INCHUSDT',
   'AAVEUSDT',
   'ADAUSDC',
   'ADAUSDT',
   'AKROUSDT',
   'ALGOUSDT',
   'ANKRUSDT',
   'ARPAUSDT',
   'ATOMUSDT',
   'AVAUSDT',
   'AVAXUSDT',
   'BATUSDT',
   'BCHSVUSDC',
   'BCHSVUSDT',
   'BCHUSDC',
   'BCHUSDT',
   'BNBUSDT',
   'BONDUSDT',
   'BTCDAI',
   'BTCPAX',
   'BTCTUSD',
   'BTCUSDC',
   'BTCUSDT',
   'BTTUSDT',
   'CAKEUSDT',
   'CELOUSDT',
   'CHRUSDT',
   'CHZUSDT',
   'CKBUSDT',
   'COMPUSDT',
   'COTIUSDT',
   'CRVUSDT',
   'DASHUSDT',
   'DEGOUSDT',
   'DGBUSDT',
   'DIAUSDT',
   'DODOUSDT',
   'DOGEUSDC',
   'DOGEUSDT',
   'DOTUSDT',
   'ENJUSDT',
   'EOSUSDC',
   'EOSUSDT',
   'ETCUSDT',
   'ETHDAI',
   'ETHPAX',
   'ETHTUSD',
   'ETHUSDC',
   'ETHUSDT',
   'FILUSDT',
   'FORTHUSDT',
   'FTMUSDT',
   'GRTUSDT',
   'ICPUSDT',
   'IOSTUSDT',
   'JSTUSDT',
   'KSMUSDT',
   'LINKUSDC',
   'LINKUSDT',
   'LPTUSDT',
   'LRCUSDT',
   'LTCUSDC',
   'LTCUSDT',
   'LUNAUSDT',
   'MANAUSDT',
   'MASKUSDT',
   'MATICUSDT',
   'MIRUSDT'

### Download Data for the last lookback period

In [23]:
end = datetime.now(timezone.utc)
start = end - LOOKBACK_PERIOD

In [24]:
db_storage = InfluxDbStorage()
for exchange_pair in EXCHANGE_PAIRS:
    exchange1 = exchange_pair[0]
    exchange2 = exchange_pair[1]

    common_pairs = common_pairs_dict[exchange1][exchange2]
    downloaded = 0
    to_download = 2 * len(common_pairs)
    for common_pair in common_pairs:
        exchange1_asset = Asset(symbol=common_pair, exchange=exchange1)
        DATA_HANDLERS[exchange1].save_ticker_data_in_db_storage(
            exchange1_asset, db_storage, start, end
        )

        exchange2_asset = Asset(symbol=common_pair, exchange=exchange2)
        DATA_HANDLERS[exchange2].save_ticker_data_in_db_storage(
            exchange2_asset, db_storage, start, end
        )
        downloaded += 2
        print(f"Progress: {downloaded} / {to_download}")

Progress: 2 / 228
Progress: 4 / 228
Progress: 6 / 228
Progress: 8 / 228
Progress: 10 / 228
Progress: 12 / 228
Progress: 14 / 228
Progress: 16 / 228
Progress: 18 / 228
Progress: 20 / 228
Progress: 22 / 228
Progress: 24 / 228
Progress: 26 / 228
Progress: 28 / 228
Progress: 30 / 228
Progress: 32 / 228
Progress: 34 / 228
Progress: 36 / 228
Progress: 38 / 228
Progress: 40 / 228
Progress: 42 / 228
Progress: 44 / 228
Progress: 46 / 228
Progress: 48 / 228
Progress: 50 / 228
Progress: 52 / 228
Progress: 54 / 228
Progress: 56 / 228
Progress: 58 / 228
Progress: 60 / 228
Progress: 62 / 228
Progress: 64 / 228
Progress: 66 / 228
Progress: 68 / 228
Progress: 70 / 228
Progress: 72 / 228
Progress: 74 / 228
Progress: 76 / 228
Progress: 78 / 228
Progress: 80 / 228
Progress: 82 / 228
Progress: 84 / 228
Progress: 86 / 228
Progress: 88 / 228
Progress: 90 / 228
Progress: 92 / 228
Progress: 94 / 228
Progress: 96 / 228
Progress: 98 / 228
Progress: 100 / 228
Progress: 102 / 228
Progress: 104 / 228
Progress: 106

In [48]:
initial_budget = 2000
errors_pct_dict = {}
profit_pct_dict = {}
profit_pct_corrected_dict = {}
profit_rank = {}


for exchange_pair in EXCHANGE_PAIRS:
    exchange1 = exchange_pair[0]
    exchange2 = exchange_pair[1]
    exchanges = [exchange1, exchange2]
    print(f"Checking arbitrage opportunities for {exchange1} and {exchange2} exchange pair")
    
    common_pairs = common_pairs_dict[exchange1][exchange2]
    common_pairs = [
     'BTCDAI',
     'XMRUSDT',
     'ENJUSDT',
     'ETHDAI',
     'SNXUSDT',
     'ETHTUSD',
     'MIRUSDT',
     'COMPUSDT',
     'BONDUSDT',
     '1INCHUSDT',
     'BTCTUSD',
     'ZENUSDT',
     'WAVESUSDT',
     'FORTHUSDT',
     'NANOUSDT'
    ]
    to_process = len(common_pairs)
    processed = 0
    for common_pair in common_pairs:
        common_pair_key = f"{exchange1}_{exchange2}_{common_pair}"
        print(f"Checking arbitrage opportunities for {common_pair}")
        events = deque()
        assets = [Asset(symbol=common_pair, exchange=exchange) for exchange in exchanges]
        assets_dict = {asset.exchange: asset for asset in assets}

        # load data
        #feed: Feed = CsvFeed(
        #    {
        #        asset: f"backtests/crypto_arbitrage/data/{common_pair.lower()}_{asset.exchange.lower()}.csv"
        #        for asset in assets
        #    },
        #    events,
        #)
        
        feed: Feed = ExternalStorageFeed(
            assets = assets,
            events = events,
            time_unit = timedelta(minutes=1),
            start = start,
            end = end,
            db_storage = db_storage,
            file_storage = None,
            market_cal = None,
        )
            
        # Check wether data is empty or not
        exchange1_candle_dataframe = feed.candle_dataframes[
            assets_dict[exchange1]
        ]
        exchange2_candle_dataframe = feed.candle_dataframes[
            assets_dict[exchange2]
        ]
        
        if exchange1_candle_dataframe.empty or exchange2_candle_dataframe.empty:
            processed += 1
            print(f"{common_pair_key} data is empty for at least one of the exchange so it is skipped.")
            print(f"Current rank: {profit_rank}")
            print(f"Progress: {processed} / {to_process}")
            continue
        
        # don't process the pair if the volume is low
        exchange1_prices = pd.to_numeric(exchange1_candle_dataframe["close"])
        exchange1_volumes = pd.to_numeric(exchange1_candle_dataframe["volume"])
        exchange1_avg_price = exchange1_prices.median()
        print(f"price = {exchange1_avg_price}")
        exchange1_avg_volume = exchange1_volumes.median()
        print(f"volume = {exchange1_avg_volume}")

        exchange2_prices = pd.to_numeric(exchange2_candle_dataframe["close"])
        exchange2_volumes = pd.to_numeric(exchange2_candle_dataframe["volume"])
        exchange2_avg_price = exchange2_prices.median()
        exchange2_avg_volume = exchange2_volumes.median()
        
        price = min(exchange1_avg_price, exchange2_avg_price)
        volume = min(exchange1_avg_volume, exchange2_avg_volume)
        
        volume_in_cash = volume * price
        
        if volume_in_cash < MINIMUM_VOLUME_IN_CASH:
            processed += 1
            print(f"{common_pair_key} volume in cash {volume_in_cash} is lower than the minimum volume in cash required {MINIMUM_VOLUME_IN_CASH} so it is skipped.")
            print(f"Current rank: {profit_rank}")
            print(f"Progress: {processed} / {to_process}")
            continue
            
        # Create brokers for exchanges, put a big amount of cash and a big amount of shares to allow all two
        # ways transactions
        strategies = {ArbitrageStrategy: [{"margin_factor": 2}]}
        
        clock = SimulatedClock()
        initial_funds = initial_budget / 2
        # print(f"initial_funds = {initial_funds}")
        # Create brokers with a big amount of money
        exchange1_broker = SimulatedBroker(
            clock,
            events,
            initial_funds=initial_funds,
            fee_model=FEE_MODELS[exchange1],
            exchange=exchange1,
        )
        exchange1_broker.subscribe_funds_to_portfolio(initial_funds)
        exchange1_first_candle = exchange1_candle_dataframe.get_candle(0)
        exchange1_broker.update_price(exchange1_first_candle)

        exchange2_broker = SimulatedBroker(
            clock,
            events,
            initial_funds=initial_funds,
            fee_model=FEE_MODELS[exchange2],
            exchange=exchange2,
        )
        # exchange2_broker.subscribe_funds_to_portfolio(initial_funds)
        exchange2_first_candle = exchange2_candle_dataframe.get_candle(0)
        exchange2_broker.update_price(exchange2_first_candle)
        max_size_exchange1 = exchange1_broker.max_entry_order_size(
            assets_dict[exchange1], Direction.LONG, initial_funds
        )
        max_size_exchange2 = exchange2_broker.max_entry_order_size(
            assets_dict[exchange2], Direction.LONG, initial_funds
        )
        initial_size = max_size_exchange2
        
        # exchange 2
        candle = Candle(
            asset=assets_dict[exchange2], open=0, high=0, low=0, close=0, volume=0
        )
        exchange2_broker.update_price(candle)
        order = Order(
            asset=assets_dict[exchange2],
            action=Action.BUY,
            direction=Direction.LONG,
            size=initial_size,
            signal_id="0",
            limit=None,
            stop=None,
            target=None,
            stop_pct=None,
            type=OrderType.MARKET,
            clock=clock,
            time_in_force=timedelta(minutes=5),
        )
        exchange2_broker.execute_market_order(order)
        
        # prepare event loop parameters
        broker_manager = BrokerManager(
            brokers={
                exchange1: exchange1_broker,
                exchange2: exchange2_broker,
            },
            clock=clock,
        )
        position_sizer = PositionSizer(broker_manager=broker_manager, integer_size=False)
        order_creator = OrderCreator(broker_manager=broker_manager)
        order_manager = OrderManager(
            events=events,
            broker_manager=broker_manager,
            position_sizer=position_sizer,
            order_creator=order_creator,
        )
        indicators_manager = IndicatorsManager(
            preload=True, initial_data=feed.candles
        )
        event_loop = EventLoop(
            events=events,
            assets=assets,
            feed=feed,
            order_manager=order_manager,
            strategies_parameters=strategies,
            indicators_manager=indicators_manager,
            close_at_end_of_day=False,
            close_at_end_of_data=False
        )

        # get initial state of portfolio for stats computation total_market_value
        exchange1_broker.update_price(exchange1_first_candle)
        exchange2_broker.update_price(exchange2_first_candle)
        initial_market_values = sum([broker_manager.get_broker(exchange).get_portfolio_total_market_value() for exchange in exchanges])
        initial_cash_balances = sum([broker_manager.get_broker(exchange).get_portfolio_cash_balance() for exchange in exchanges])
        initial_equities = initial_cash_balances + initial_market_values
        
        # print(f"initial_market_values = {initial_market_values}")
        # print(f"initial_cash_balances = {initial_cash_balances}")
        # print(f"initial_equities = {initial_equities}")

        event_loop.loop()
        
        exchange1_history = exchange1_broker.portfolio.history
        exchange1_transactions = [portfolio_event for portfolio_event in exchange1_history if portfolio_event.type == "symbol_transaction"]
        # remove first transaction, which is to add securities in the broker
        exchange1_transactions = exchange1_transactions[1:]

        exchange2_history = exchange2_broker.portfolio.history
        exchange2_transactions = [portfolio_event for portfolio_event in exchange2_history if portfolio_event.type == "symbol_transaction"]
        # remove first transaction, which is to add securities in the broker
        exchange2_transactions = exchange2_transactions[1:]
        
        # exchange1_transactions_timestamps = np.array([transaction.timestamp for transaction in exchange1_transactions], dtype=datetime)
        # exchange2_transactions_timestamps = np.array([transaction.timestamp for transaction in exchange2_transactions], dtype=datetime)
        # common_timestamps = np.intersect1d(exchange1_transactions_timestamps, exchange2_transactions_timestamps)
        
        # filter timestamps
        # exchange1_transactions = [transaction for transaction in exchange1_transactions if transaction.timestamp in common_timestamps]
        # exchange2_transactions = [transaction for transaction in exchange2_transactions if transaction.timestamp in common_timestamps]
        
        nb_transactions1 = len(exchange1_transactions)
        nb_transactions2 = len(exchange2_transactions)
        min_nb_transactions = min(nb_transactions1, nb_transactions2)      
        if len(exchange1_transactions) < MINIMUM_TRANSACTIONS:
            processed += 1
            print(f"{common_pair_key} has only {min_nb_transactions} transactions which is less than the minimum required number of transactions {MINIMUM_TRANSACTIONS} so it is skipped.")
            print(f"Current rank: {profit_rank}")
            print(f"Progress: {processed} / {to_process}")
            continue
        
        # find missed opportunities
        
        """
        When an arbitrage opportunity is found, we submit 2 orders to the brokers of the 2 exchanges.
        If one of the order is not executed because of for example not enough cash or, whathever reason,
        we call it an "error". The less errors you have, the better it is for ensuring the stability of the strategy
        """
        nb_errors = 0
        transaction_profits = []
        i = 0
        j = 0
        while i < nb_transactions1 and j < nb_transactions2:
            transaction1 = exchange1_transactions[i]
            transaction2 = exchange2_transactions[j]
            if transaction1.timestamp == transaction2.timestamp:
                i += 1
                j += 1
            elif transaction1.timestamp < transaction2.timestamp:
                i += 1
                nb_errors += 1
                continue
            else: # transaction1.timestamp > transaction2.timestamp
                j += 1
                nb_errors += 1
                continue
            if transaction1.action == Action.BUY:
                transaction_profit = transaction2.credit - transaction1.debit
            else:
                transaction_profit = transaction1.credit - transaction2.debit
            # print(f"{transaction1.timestamp} - profit = {transaction_profit}")
            transaction_profits.append(transaction_profit)
        
        if nb_errors > min_nb_transactions:
            processed += 1
            print(f"{common_pair_key} nb errors is greater than number of transactions {min_nb_transactions} transactions so it is skipped.")
            print(f"Current rank: {profit_rank}")
            print(f"Progress: {processed} / {to_process}")
            continue

        
        errors_pct = nb_errors / max(nb_transactions1, nb_transactions2) * 100
        cash_profit = sum(transaction_profits)
        print(f"cash_profit = {cash_profit}")

        # let's find the coefficient of variation to filter
        cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
        coefficient_of_variation = cv(transaction_profits)
        
        get_or_create_nested_dict(errors_pct_dict, common_pair_key)
        errors_pct_dict[common_pair_key] = errors_pct
        
        final_market_values = sum([broker_manager.get_broker(exchange).get_portfolio_total_market_value() for exchange in exchanges])
        final_cash_balances = sum([broker_manager.get_broker(exchange).get_portfolio_cash_balance() for exchange in exchanges])
        final_equities = final_market_values + final_cash_balances

        # print(f"final_market_values = {final_market_values}")
        # print(f"final_cash_balances = {final_cash_balances}")
        # print(f"final_equities = {final_equities}")

        profit = final_equities - initial_equities

        profit_pct = profit / initial_equities * 100
        
        cash_profit_pct = cash_profit / initial_cash_balances * 100

        get_or_create_nested_dict(profit_pct_dict, common_pair_key)
        profit_pct_dict[common_pair_key] = {
            "profit_pct": profit_pct,
            "cash_profit_pct": cash_profit_pct,
            "coefficient of varition (abs)": abs(coefficient_of_variation),
            "nb_errors": nb_errors,
            "errors_pct": errors_pct,
            "minimum number of transactions in both exchanges": min_nb_transactions
        }
        
        # exchange1_broker.portfolio.history_to_df().to_csv(f"backtests/crypto_arbitrage/data/transactions/{common_pair.lower()}_{exchange1.lower()}_transactions.csv")
        # exchange2_broker.portfolio.history_to_df().to_csv(f"backtests/crypto_arbitrage/data/transactions/{common_pair.lower()}_{exchange2.lower()}_transactions.csv")
        
        processed += 1
        print(f"{common_pair_key} profit: {profit_pct}")
        
        # Order by number of errors
        profit_rank = {k: v for k, v in sorted(profit_pct_dict.items(), key=lambda item: item[1]["coefficient of varition (abs)"], reverse=False)}
        print(f"Current rank: {profit_rank}")
        print(f"Progress: {processed} / {to_process}")

Checking arbitrage opportunities for BINANCE and KUCOIN exchange pair
Checking arbitrage opportunities for BTCDAI
price = 33870.86
volume = 0.0
BINANCE_KUCOIN_BTCDAI volume in cash 0.0 is lower than the minimum volume in cash required 1000 so it is skipped.
Current rank: {}
Progress: 1 / 15
Checking arbitrage opportunities for XMRUSDT
price = 213.66
volume = 30.858710000000002
BINANCE_KUCOIN_XMRUSDT volume in cash 0.0 is lower than the minimum volume in cash required 1000 so it is skipped.
Current rank: {}
Progress: 2 / 15
Checking arbitrage opportunities for ENJUSDT
price = 1.2072500000000002
volume = 11806.720000000001
BINANCE_KUCOIN_ENJUSDT volume in cash 175.507383858828 is lower than the minimum volume in cash required 1000 so it is skipped.
Current rank: {}
Progress: 3 / 15
Checking arbitrage opportunities for ETHDAI
price = 2231.75
volume = 1.215965
BINANCE_KUCOIN_ETHDAI volume in cash 0.0 is lower than the minimum volume in cash required 1000 so it is skipped.
Current rank: {}


In [50]:
best_results_df = pd.DataFrame.from_dict(profit_rank, orient="index")

In [51]:
best_results_df

,profit_pct,cash_profit_pct,coefficient of varition (abs),nb_errors,errors_pct,minimum number of transactions in both exchanges
BINANCE_KUCOIN_SNXUSDT,24.169861,-0.836494,99.636606,2,4.081633,49
BINANCE_KUCOIN_COMPUSDT,19.869630,-0.241172,174.598381,3,13.043478,22


In [36]:
best_results_df.corr(method='pearson')

,profit_pct,cash_profit_pct,coefficient of varition (abs),nb_errors,minimum number of transactions in both exchanges
profit_pct,1.000000,0.100477,-0.261412,0.406786,0.279019
cash_profit_pct,0.100477,1.000000,-0.118749,0.350913,0.931028
coefficient of varition (abs),-0.261412,-0.118749,1.000000,0.335351,0.051262
nb_errors,0.406786,0.350913,0.335351,1.000000,0.631731
minimum number of transactions in both exchanges,0.279019,0.931028,0.051262,0.631731,1.000000
